In [18]:
import os
import numpy as np
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image

In [19]:
# Mendefinisikan direktori dataset asli dan dataset hasil deteksi tepi
dataset_awal = "Dataset/"
dataset_morph = "dataset_morph"

# Memeriksa apakah folder dataset hasil sudah ada, jika tidak buat folder baru
if not os.path.exists(dataset_morph):
    os.makedirs(dataset_morph)

# Mendapatkan daftar nama file gambar dalam dataset awal
daftar_file = os.listdir(dataset_awal)

In [20]:
print(daftar_file)

['Sedan', 'Sport', 'SUV']


In [21]:
# Fungsi gray
def Grayscale (image):
    ker_gauss = np.ones((3,3))/9
    image0 = image[:,:,0]
    image1 = image[:,:,1]
    image2 = image[:,:,2]
    filteredimage0=cv.filter2D(image0,-1,ker_gauss)
    filteredimage1=cv.filter2D(image1,-1,ker_gauss)
    filteredimage2=cv.filter2D(image2,-1,ker_gauss)
    return filteredimage0/3 + filteredimage1/3 + filteredimage2/3

In [22]:
# Fungsi Resize
def resize (image):
    res = cv.resize(image, (300, 300), interpolation=cv.INTER_NEAREST)
    return res

In [23]:
# Fungsi Threshold
def Threshold(image):
    matriks = np.array(image)
    result = np.zeros(matriks.shape).astype(int)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if matriks[i,j] <= 128 : 
                result[i,j] = 255
            else:
                result[i,j] = 0

    return result

In [24]:
# Fungsi Morph
def Grad_Morph(citra):
  # Konversi citra ke grayscale jika belum grayscale
  if len(citra.shape) > 2:
    citra = cv.cvtColor(citra, cv.COLOR_BGR2GRAY)

  # Hitung gradien horizontal dan vertikal
  gx = cv.Sobel(citra, cv.CV_64F, 1, 0)
  gy = cv.Sobel(citra, cv.CV_64F, 0, 1)

  # Hitung magnitudo gradien
  magnitudo = np.sqrt(gx**2 + gy**2)

  # Non-maximum suppression (NMS)
  angkaran = np.arctan2(gy, gx)
  sudut = np.rad2deg(angkaran) + 90

  sudut_kuantisasi = np.zeros_like(sudut)
  sudut_kuantisasi[sudut < 0] = 180
  sudut_kuantisasi[np.where(sudut >= 0) & (sudut < 45)] = 0
  sudut_kuantisasi[np.where(sudut >= 45) & (sudut < 90)] = 45
  sudut_kuantisasi[np.where(sudut >= 90) & (sudut < 135)] = 90
  sudut_kuantisasi[np.where(sudut >= 135) & (sudut < 180)] = 135

  arah_tepi = sudut_kuantisasi.astype(np.int32)

  citra_tepi = np.zeros_like(magnitudo)

  for i in range(citra.shape[0]):
    for j in range(citra.shape[1]):
      sudut_saat_ini = sudut_kuantisasi[i, j]
      arah_tepi_tetangga = arah_tepi[i, j - 1] if j > 0 else 0
      arah_tepi_tetangga_kanan = arah_tepi[i, j + 1] if j < citra.shape[1] - 1 else 0
      arah_tepi_tetangga_atas = arah_tepi[i - 1, j] if i > 0 else 0
      arah_tepi_tetangga_bawah = arah_tepi[i + 1, j] if i < citra.shape[0] - 1 else 0

      if (sudut_saat_ini == 0 and magnitudo[i, j] > magnitudo[i, j - 1] and magnitudo[i, j] > magnitudo[i, j + 1]) or \
         (sudut_saat_ini == 45 and magnitudo[i, j] > magnitudo[i - 1, j - 1] and magnitudo[i, j] > magnitudo[i + 1, j + 1]) or \
         (sudut_saat_ini == 90 and magnitudo[i, j] > magnitudo[i - 1, j] and magnitudo[i, j] > magnitudo[i + 1, j]) or \
         (sudut_saat_ini == 135 and magnitudo[i, j] > magnitudo[i - 1, j + 1] and magnitudo[i, j] > magnitudo[i + 1, j - 1]):
        citra_tepi[i, j] = magnitudo[i, j]

  # Ambang batas
  ambang_bawah = 0.01 * np.max(magnitudo)
  ambang_atas = 0.2 * np.max(magnitudo)
  citra_tepi[magnitudo < ambang_bawah] = 0
  citra_tepi[magnitudo >= ambang_atas] = 255

  # Menghilangkan tepi yang terhubung
  citra_tepi = cv.morphologyEx(citra_tepi, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_RECT, (3, 3)))

  return citra_tepi

In [25]:
# Fungsi Normalisasi
def normalisasi(image):
    min_val = np.min(image)
    max_val = np.max(image)
    
    normalized_image = (image - min_val) / (max_val - min_val)
    uint8_image = (normalized_image * 255).astype(np.uint8)
    
    return uint8_image

In [26]:
# for nama_file in daftar_file:

#     # Membaca gambar dari dataset awal
#     gambar_awal = cv.imread(os.path.join(dataset_awal, nama_file))

#     # Mengubah gambar menjadi grayscale (Ganti pake fungsi manual)
#     gambar_grayscale = cv.cvtColor(gambar_awal, cv.COLOR_BGR2GRAY)

#     # Melakukan resize (Ganti pake fungsi manual)
#     gambar_kecil = resize (gambar_grayscale)
#     # Melakukan thresholding (Ganti pake fungsi manual)
#     _, binary = cv.threshold(gambar_kecil, 127, 255, cv.THRESH_BINARY)
#     # Mendefinisikan kernel
#     kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
#     # Mendeteksi tepi menggunakan gradient morfologi
#     gradient = cv.morphologyEx(binary, cv.MORPH_GRADIENT, kernel)

#     cv.imwrite(os.path.join(dataset_morph, nama_file), gradient)

In [27]:
# Memuat data gambar
def load_data(data_dir):
    images = []
    labels = []

    for folder in os.listdir(data_dir):
        label = folder
        for image_filename in os.listdir(os.path.join(data_dir, folder)):
            image_path = os.path.join(data_dir, folder, image_filename)
            image = Image.open(image_path).resize((32, 32))
            image_data = np.array(image)
            images.append(image_data.flatten())
            labels.append(label)

    return np.array(images), np.array(labels)

In [28]:
X, y = load_data('dataset_morph')

In [29]:
# Membagi data menjadi training dan testing (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
# Membuat model KNN
knn = KNeighborsClassifier(n_neighbors=5)

In [31]:
# Melatih model dengan data training
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [32]:
# Melakukan prediksi pada data testing
y_pred = knn.predict(X_test)

In [33]:
# Menghitung akurasi model
accuracy = np.mean(y_pred == y_test) * 100

In [34]:
# Menampilkan hasil akurasi
print("Akurasi model KNN:", accuracy, "%")

Akurasi model KNN: 38.036809815950924 %
